In [1]:
from package import *

import requests
from pprint import pprint
from functools import partial

pprint = partial(pprint, width = 200)


class CallApi:
	endpoints = {
		'videos':        'https://www.googleapis.com/youtube/v3/videos',
		'searchs':       'https://www.googleapis.com/youtube/v3/search',
		'channels':      'https://www.googleapis.com/youtube/v3/channels',
		'playlists':     'https://www.googleapis.com/youtube/v3/playlists',
		'playlistItems': 'https://www.googleapis.com/youtube/v3/playlistItems',
		'activities':    'https://www.googleapis.com/youtube/v3/activities',
		'watch':         'http://www.youtube.com/watch'
	}
	def __init__(self, **kwargs):
		pprint(dir(github))
		self.api_key = github.youtube_api_key


	def request(self, endpoint, **parameters):
		if not endpoint.endswith('s'): endpoint += 's'
		url = self.endpoints[endpoint]

		parameters['key'] = self.api_key

		response = requests.get(url, params = parameters)
		return response


test_api = CallApi()
youtube = YouTube(api_key = github.youtube_api_key)

playlist_parameters = {
	'id':         "PL1cXh4tWqmsEQPeLEJ5V3k5knt-X9k043",
	'maxResults': 50,
	'part':       "snippet,contentDetails",
}
search_parameters = {
	'playlistId': playlist_parameters['id'],
	'maxResults': 50,
	'part':       'snippet',
}
channel_parameters = {
	'id':   'UCkxctb0jr8vwa4Do6c6su0Q',
	'part': "snippet,statistics,topicDetails"
}
video_parameters = {
	'id':   'p9RKAalhtZM',
	'part': 'snippet,statistics,topicDetails'
}

response = test_api.request('channels', **channel_parameters)

#pprint(response.json(), width = 200)
print(type(response))
print("Finished!")


['DATA_FOLDER',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'github_folder',
 'os',
 'sys',
 'tabletools',
 'timetools',
 'youtube_api_key',
 'youtube_subscriptions']


<class 'requests.models.Response'>
Finished!


In [4]:
result = youtube.getPlaylistItems(playlist_parameters['id'])
for item in result:
	print(item)



ApiResponse('playlistItem', 'FrLgREKD4kk')
ApiResponse('playlistItem', 'Tihfx6DunZs')
ApiResponse('playlistItem', '5LGI2YNGPY4')
ApiResponse('playlistItem', 'C24-6jgPnKM')
ApiResponse('playlistItem', 'H3_jfbgfdvk')
ApiResponse('playlistItem', 'N0cOBlKhfxg')
ApiResponse('playlistItem', 'Nuj7D5CC_u4')
ApiResponse('playlistItem', '3E-1UMsb7wM')
ApiResponse('playlistItem', 'WpfWIizrd6U')
ApiResponse('playlistItem', 'V9Znhvbyo_c')
ApiResponse('playlistItem', 'dgVEs85Q7pg')
ApiResponse('playlistItem', 'm2gM_5j0QTI')
ApiResponse('playlistItem', 'wML52FKQyRo')
ApiResponse('playlistItem', '8yC3Jl77Ywo')
ApiResponse('playlistItem', 'u30n18MIeC0')
ApiResponse('playlistItem', 'UpwAXpA680Q')
ApiResponse('playlistItem', 'ZzmWedLiHMg')
ApiResponse('playlistItem', 'IBHK6ePF7B8')
ApiResponse('playlistItem', 'ajvP0rrlqV0')
ApiResponse('playlistItem', '0mADvdqKQOI')
ApiResponse('playlistItem', 'N0u0S5ojygo')
ApiResponse('playlistItem', 'I2XdnuZKMlM')
ApiResponse('playlistItem', 'WXA4IsJTvRo')
ApiResponse

In [5]:
pprint(item.api_response)

{'etag': '"S8kisgyDEblalhHF9ooXPiFFrkc/uSWy72D2-srPx7gdH2aNS14tyVw"',
 'id': 'UEwxY1hoNHRXcW1zRVFQZUxFSjVWM2s1a250LVg5azA0My5CNTZFOTNGQzZEODg1RUQx',
 'kind': 'youtube#playlistItem',
 'snippet': {'channelId': 'UCkxctb0jr8vwa4Do6c6su0Q',
             'channelTitle': 'LetsPlay',
             'description': 'Geoff, Jack, Michael, Gavin, Ray and Ryan all team up on a long quest to fight the Ender Dragon. Will they make it? Or will their lovely relationship buckle under the '
                            'pressure? Join FIRST to watch episodes early: http://bit.ly/2emdy6q\n'
                            '\n'
                            "» Get your Let's Play merch: http://bit.ly/2emgP5J\n"
                            '» Subscribe: http://bit.ly/1BuRgl1\n'
                            '\n'
                            'About Minecraft:\n'
                            "Tune in every week to watch a game that you'd never play yourself. It's Minecraft! And it's never gonna stop.\n"
                  

In [4]:
pprint(r.toStandard())

{'channelId': 'UCkxctb0jr8vwa4Do6c6su0Q',
 'channelName': 'LetsPlay',
 'itemDescription': 'Geoff, Jack, Michael, Gavin, Ray and Ryan all team up on a long quest to fight the Ender Dragon. Will they make it? Or will their lovely relationship buckle under the pressure? '
                    'Join FIRST to watch episodes early: http://bit.ly/2emdy6q\n'
                    '\n'
                    "» Get your Let's Play merch: http://bit.ly/2emgP5J\n"
                    '» Subscribe: http://bit.ly/1BuRgl1\n'
                    '\n'
                    'About Minecraft:\n'
                    "Tune in every week to watch a game that you'd never play yourself. It's Minecraft! And it's never gonna stop.\n"
                    '\n'
                    "More from Let's Play:\n"
                    '» Achievement Hunter: http://bit.ly/AHYTChannel\n'
                    '\n'
                    "Let's Play Minecraft: Ep. 49 - THE END Part 1\n"
                    'https://www.youtube.com/user/l

In [ ]:
def getChannelSeries(channel):
	"""
		Determines which videos in the channel belong to a series.
	Parameters
	----------
	channel

	Returns
	-------

	"""
	videos = channel.videos
	
	